In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as thdat
import time


In [ ]:
A=torch. ones(3)
A.shape

AA=torch.cat((A,A,A))
print(AA)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [ ]:
#Make gradient ? not sure why this works
#theowolf code https://github.com/TheodoreWolf/pinns/blob/main/src/cooling/temp_pred.ipynb

def grad(outputs, inputs):
    """Computes the partial derivative of
    an output with respect to an input.
    Args:
        outputs: (N, 1) tensor
        inputs: (N, D) tensor
    """
    return torch.autograd.grad(
        outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)


#Make an NN
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class YWTry(torch.nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        epochs=1000,
        #loss=nn.MSELoss(),
        lr=1e-4,
        #loss2=None,
        loss2_weight=0.1,
    ): #-> None:
        super().__init__()

        self.epochs = epochs
        #self.loss = loss
        self.loss2_weight = loss2_weight
        self.lr = lr

        self.layers = nn.Sequential(
            nn.Linear(input_dim, input_dim), #the linear transform part https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
            nn.Tanh(), #the activation func I changed to tanh
            nn.Linear(input_dim, input_dim),
            nn.Tanh(),
            nn.Linear(input_dim, input_dim),
            nn.Tanh(),
            nn.Linear(input_dim, input_dim),
            nn.Tanh(),
        )
        self.out = nn.Linear(input_dim, output_dim) #the final transform

    def forward(self, x):
        h = self.layers(x) #it gives you the input as transformed by the hidden layers of the neural networks
        out = self.out(h) #prints out the final product. 'out' as tensor
        return out


    def Train(self, X, y):

        start=time.time()
        optimiser = optim.Adam(self.parameters(), lr=self.lr)
        self.train() #https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch

        def loss_g(inputs, outputs):
          loss=(inputs-outputs)**2
          return loss.mean()

        def loss_f(input):
          U=self.Model(input)
          a=grad(U,input)[0] #because it returns you a tuple
          Ut=a[0]
          Ux=a[1]
          #Uxx=torch.autograd.grad(Ux,input[0])
          return (Ut-U*Ux)**2
              # U_partials=torch.autograd.grad(nnpred, inputs, grad_outputs=torch.ones_like(nnpred), retain_graph=True, create_graph=True)[0] #grad against inputs
              # Ut=U_partials[:,1] #grad against the time component, all rows, 2nd column
              # Ux=U_partials[:,0] #grad against space TAKES ONLY TENSORS as inputs
              # eq = (Ut-nnpred*Ux)**2 #1D burgers\

        for ep in range(self.epochs):
            optimiser.zero_grad()
            outputs = self.forward(X)
            i=random.randrange(2*N)
            loss = loss_g(y, outputs) + loss_f(cpts[i])
            #print(i)
            loss.backward()
            optimiser.step()
            if ep % int(self.epochs / 10) == 0:
                print(f"Epoch {ep}/{self.epochs}, loss: {loss.item():.2f}")
                print('time elapsed: {}'.format(time.time()-start))
        return ()

    def Model(self, X):
        self.eval()
        out = self.forward(X)
        return out #.detach().cpu().numpy()



In [ ]:
#https://pytorch.org/docs/stable/generated/torch.autograd.grad.html
#autograd only takes against scalar products
x=torch.tensor([1,2,3], dtype=float, requires_grad=True)
y=torch.tensor([1,1,1], dtype=float, requires_grad=True)
z=torch.zeros(3)
z[0]=x[0]+x[2]
z[1]=x[1]*2
z[2]=x[2]*3


# z=2*x+y
#z.backward()
#Z0_x= torch.autograd.grad(z,x)[0]
def func(x):
  z = torch.zeros(3)
  z[0] = x[0]+x[2]
  z[1] = x[1]*2
  z[2] = x[2]*3
  return z
# https://pytorch.org/docs/stable/generated/torch.autograd.functional.jacobian.html
Z0_x = torch.autograd.functional.jacobian(func, x, create_graph=False, strict=False, vectorize=False, strategy='reverse-mode')
print(Z0_x)
type(Z0_x)


tensor([[1., 0., 1.],
        [0., 2., 0.],
        [0., 0., 3.]], dtype=torch.float64)


torch.Tensor

In [ ]:
print(type(cpts[3]))
a=torch.tensor([1,1])
a.shape
a[0]

<class 'torch.Tensor'>


tensor(1)

In [ ]:
#test data
N=100
import random

#initial condition, t=0
inputinit = torch.column_stack((torch.tensor([random.random() for i in range(N)]), torch.zeros(N)))
print(inputinit.shape)

#boundary condition, x=0, x=1, t is random in [0,1]
k=torch.hstack((torch.zeros(N), torch.ones(N)))
inputboundary = torch.column_stack((k, torch.tensor([random.random() for i in range(2*N)])))

#output for initial condition
outputinit=torch.sin(np.pi*inputinit[:,0])
print(outputinit.shape)
#print(outputinit)
#output for no-penetrations conditions on boundary:
outputboundary= torch.zeros(2*N)
#print(outputboundary.shape)

#collocation pts to check physics loss
cpts = torch.column_stack((torch.tensor([random.random() for i in range(2*N)], requires_grad=True), torch.tensor([random.random() for i in range(2*N)], requires_grad=True)))
#print(cpts)
#print(cpts[3])
print(type(cpts[3]))
#Stick together (concatenate: along the same dim)
inputs =torch.cat((inputinit, inputboundary))
outputs=torch.cat((outputinit, outputboundary))

#print(inputs, outputs)


torch.Size([100, 2])
torch.Size([100])
<class 'torch.Tensor'>


In [ ]:
#Results
samples = torch.column_stack((torch.tensor([random.random() for i in range(10*N)]), torch.tensor([random.random() for i in range(10*N)])))
samplemodel=YWTry(2,1, epochs=30000, lr=5e-5).to(DEVICE)
samplemodel.Train(inputs, outputs)

Epoch 0/30000, loss: 0.20
time elapsed: 0.009164810180664062
Epoch 3000/30000, loss: 0.11
time elapsed: 11.652799844741821
Epoch 6000/30000, loss: 0.11
time elapsed: 22.364651203155518
Epoch 9000/30000, loss: 0.11
time elapsed: 33.31175208091736
Epoch 12000/30000, loss: 0.11
time elapsed: 44.25128674507141
Epoch 15000/30000, loss: 0.11
time elapsed: 55.18576502799988
Epoch 18000/30000, loss: 0.11
time elapsed: 65.6930046081543
Epoch 21000/30000, loss: 0.11
time elapsed: 77.58977150917053
Epoch 24000/30000, loss: 0.11
time elapsed: 88.72513246536255
Epoch 27000/30000, loss: 0.11
time elapsed: 99.91305303573608


()

In [ ]:
samplemodel.Model(torch.tensor([.5,.5]))

tensor([0.2272], grad_fn=<ViewBackward0>)

///divider

try1:
epoch30000, lr1e-5,optimizer=adam,
Epoch 0/30000, loss: 0.39
Epoch 3000/30000, loss: 0.32
Epoch 6000/30000, loss: 0.27
Epoch 9000/30000, loss: 0.23
Epoch 12000/30000, loss: 0.19
Epoch 15000/30000, loss: 0.16
Epoch 18000/30000, loss: 0.14
Epoch 21000/30000, loss: 0.13
Epoch 24000/30000, loss: 0.13
Epoch 27000/30000, loss: 0.13


try2: epoch30000, lr1e-4,optimizer=adam,
Epoch 0/30000, loss: 0.20
Epoch 3000/30000, loss: 0.12
Epoch 6000/30000, loss: 0.12
Epoch 9000/30000, loss: 0.12
Epoch 12000/30000, loss: 0.12
Epoch 15000/30000, loss: 0.12
Epoch 18000/30000, loss: 0.12
Epoch 21000/30000, loss: 0.12
Epoch 24000/30000, loss: 0.12
Epoch 27000/30000, loss: 0.12


try3: same as above (but i added time stamps)
usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([300, 1])) that is different to the input size (torch.Size([300])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 0/30000, loss: 0.84
time elapsed: 0.008864164352416992
Epoch 3000/30000, loss: 0.14
time elapsed: 12.571324110031128
Epoch 6000/30000, loss: 0.13
time elapsed: 23.610451221466064
Epoch 9000/30000, loss: 0.13
time elapsed: 34.72315692901611
Epoch 12000/30000, loss: 0.13
time elapsed: 45.22088980674744
Epoch 15000/30000, loss: 0.13
time elapsed: 56.28272533416748
Epoch 18000/30000, loss: 0.13
time elapsed: 67.95624613761902
Epoch 21000/30000, loss: 0.13
time elapsed: 79.10966062545776
Epoch 24000/30000, loss: 0.13
time elapsed: 90.2882821559906
Epoch 27000/30000, loss: 0.13
time elapsed: 101.59162855148315
()



try4: I added the loss_g b/c I was getting the user warning for the array sizes

Also lr=5e-5

Tweaked the physics loss to correctly compute Ut and Ux at any given input. For try 4 I tried to average over all the collocation pts, but WHY DOESN'T GRAD CALCULATE JACOBIAN AS A MULTIDIMERSIONAL TENSOR??. It only lets me compute the gradient at ONE COLOCATION POINT at one time. So then I had to sample it at random colocation points.

Epoch 0/30000, loss: 0.20
time elapsed: 0.009164810180664062
Epoch 3000/30000, loss: 0.11
time elapsed: 11.652799844741821
Epoch 6000/30000, loss: 0.11
time elapsed: 22.364651203155518
Epoch 9000/30000, loss: 0.11
time elapsed: 33.31175208091736
Epoch 12000/30000, loss: 0.11
time elapsed: 44.25128674507141
Epoch 15000/30000, loss: 0.11
time elapsed: 55.18576502799988
Epoch 18000/30000, loss: 0.11
time elapsed: 65.6930046081543
Epoch 21000/30000, loss: 0.11
time elapsed: 77.58977150917053
Epoch 24000/30000, loss: 0.11
time elapsed: 88.72513246536255
Epoch 27000/30000, loss: 0.11
time elapsed: 99.91305303573608

In [ ]:
#visualazation
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5), dpi=300)
# input shape
n_temporal, n_spatial = 10, 2000
# combine (x, t) as a vector
v = np.zeros([n_spatial, 2]) # v[:, 0] = t, v[:, 1] = x
v[:, 1] = np.linspace(-1, +1, n_spatial)
# change t
for i in range(n_temporal):
    v[:, 0] = i/n_temporal
    plt.plot(v[:, 1], samplemodel.Model(torch.tensor(v)), label=f"t = {i/N:.2f}", lw=0.75)

plt.legend(loc="upper right", fontsize=5)
plt.xlim(-1, +1)
plt.ylim(-1, +1)
plt.ylabel(f"u(t, x)")
plt.xlabel(f"x")
plt.tight_layout()
plt.savefig("u-constant-time.png")
plt.show()

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

<Figure size 1500x1500 with 0 Axes>

In [ ]:
#define losses (not necessary?)
#MS loss on boundary conditions: from princeton tutorial
def loss_g(t_p, t_c):
 squared_diffs = (t_p - t_c)**2
 return squared_diffs.mean()

#physics loss
def loss_f(inputs): #model is the neural network. I need to figure out the syntax
#should put cpts, the colocation pts, as the input
  nnpred=Model(inputs)
  Ut=grad(nnpred, inputs[:,1]) #grad against the time component, all rows, 2nd column
  Ux=grad(nnpred, inputs[:,1])
  eq = (Ut-nnpred*Ux)**2
  return eq.mean()

#theowolf code
def physics_loss(model: torch.nn.Module):
    ts = torch.linspace(0, 1000, steps=1000,).view(-1,1).requires_grad_(True).to(DEVICE)
    temps = model(ts)
    dT = grad(temps, ts)[0]
    pde = R*(Tenv - temps) - dT

    return torch.mean(pde**2)

In [ ]:
class YWModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
